In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
class MF(object):
    def __init__(self, Y, K, lam=0.1, Xinit=None, Winit=None, learning_rate=0.5, max_iter=1000, print_every=100):
        self.Y = Y
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.n_users = int(np.max(self.Y[:, 0])) + 1
        self.n_items = int(np.max(self.Y[:, 1])) + 1
        self.n_ratings = Y.shape[0]
        self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit
        self.W = np.random.randn(K, self.n_users) if Winit is None else Winit
        self.b = np.random.randn(self.n_items)
        self.d = np.random.randn(self.n_users)

    def loss(self):
        L = 0
        for i in range(self.n_ratings):
            n = int(self.Y[i, 0])
            m = int(self.Y[i, 1])
            rating = self.Y[i, 2]
            L += 0.5*(self.X[m].dot(self.W[:, n]) +
                      self.d[n] + self.b[m] - rating)**2

        L /= self.n_ratings
        # regularization
        return L + 0.5*self.lam*(np.sum(self.W**2) + np.sum(self.X**2))

    def updateXb(self):
        for m in range(self.n_items):
            # get all users who rated item m and corresponding ratings
            ids = np.where(self.Y[:, 1] == m)[0]
            user_ids = self.Y[ids, 0].astype(np.int32)
            ratings = self.Y[ids, 2]
            Wm = self.W[:, user_ids]
            dm = self.d[user_ids]
            for i in range(30):
                xm = self.X[m]
                error = xm.dot(Wm) + self.b[m] + dm - ratings
                grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
                grad_bm = np.sum(error)/self.n_ratings
                # gradient descent
                self.X[m] -= self.learning_rate*grad_xm.reshape(-1)
                self.b[m] -= self.learning_rate*grad_bm

    def updateWd(self):
        for n in range(self.n_users):
            # get all items rated by user n, and the corresponding ratings
            ids = np.where(self.Y[:, 0] == n)[0]
            item_ids = self.Y[ids, 1].astype(np.int32)
            ratings = self.Y[ids, 2]
            Xn = self.X[item_ids]
            bn = self.b[item_ids]
            for i in range(30):
                wn = self.W[:, n]
                error = Xn.dot(wn) + bn + self.d[n] - ratings
                grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
                grad_dn = np.sum(error)/self.n_ratings
                # gradient descent
                self.W[:, n] -= self.learning_rate*grad_wn.reshape(-1)
                self.d[n] -= self.learning_rate*grad_dn

    def fit(self):
        for it in range(self.max_iter):
            self.updateWd()
            self.updateXb()
            if (it+1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y)
                print('iter = %d, loss = %.4f, RMSE train = %.4f' %
                      (it + 1, self.loss(), rmse_train))

    def pred(self, u, i):
        u, i = int(u), int(i)
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u]
        return max(0, min(5, pred))

    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2
        RMSE = np.sqrt(SE/n_tests)
        return RMSE

## MovieLens

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [5]:
rs = MF(rate_train, K = 50, lam = .01, print_every = 5, learning_rate = 50, max_iter = 30)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nMatrix Factorization CF, RMSE = %.4f' %RMSE)

iter = 5, loss = 0.4446, RMSE train = 0.9427
iter = 10, loss = 0.4217, RMSE train = 0.9182
iter = 15, loss = 0.4176, RMSE train = 0.9137
iter = 20, loss = 0.4163, RMSE train = 0.9123
iter = 25, loss = 0.4157, RMSE train = 0.9116
iter = 30, loss = 0.4154, RMSE train = 0.9112

Matrix Factorization CF, RMSE = 0.9619
